# Notebook - Custom Pipeline

by Simon Böck

The structure in general is kept very simple:
- Feed the Speech-to-text API by Google Cloud with the corresponding audio file
- Feed the DeepL API with the correspoinding output of the Speech-to-text API of Google Cloud

In [1]:
#pip install --upgrade google-cloud-speech google-cloud-translate

In [2]:
# pip install --upgrade google-cloud-storage

In [3]:
import os
import subprocess
import ffmpeg
import requests
from google.cloud import speech, storage

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'your_key.json'

In [5]:
client = speech.SpeechClient()

The corresponding audio file is uploaded to the Google Cloud Storage, due to its size. Otherwise it would not be able to run that smoothly. The upload and creation of such bucket, in which you can store the files needed is very simple and can looked up here https://cloud.google.com/storage?hl=en.

In [6]:
# Upload the large audio file to Google Cloud Storage
storage_client = storage.Client()

bucket = storage_client.get_bucket('bucket-name')  # Replace with your bucket name

#Prepare the URI for the audio file in Google Cloud Storage
gcs_uri = 'gs://bucket-name/file.wav'

# Configure the audio file's settings for the recognition request
long_running_recognize_config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="de-DE",
    audio_channel_count=2
)

audio = speech.RecognitionAudio(uri=gcs_uri)

# Asynchronously transcribe the audio
operation = client.long_running_recognize(config=long_running_recognize_config, audio=audio)

print('Waiting for operation to complete...')
response = operation.result(timeout=600)  # Adjust timeout as needed

# Open a file to write the transcriptions
with open('transcription_output.txt', 'w') as file:
    for result in response.results:
        file.write(result.alternatives[0].transcript + '\n')


In [7]:
def translate_text_with_deepl(text_file, api_key):

    try:
        # Attempt to read the file with UTF-8 encoding
        with open(text_file, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        # If UTF-8 fails, try reading with ISO-8859-1 encoding
        with open(text_file, 'r', encoding='iso-8859-1') as file:
            text = file.read()

    url = "https://api-free.deepl.com/v2/translate"
    headers = {"Authorization": f"DeepL-Auth-Key {api_key}"}
    data = {
        "text": text,
        "target_lang": "EN",
        "source_lang": "DE"
    }

    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        json_response = response.json()
        translations = json_response.get('translations', [])
        if translations:
            translated_text = translations[0].get('text', 'No translation found.')
            # Save the translated text to a file
            output_file = 'translated_output.txt'
            with open(output_file, 'w', encoding='utf-8') as out_file:
                out_file.write(translated_text)
            return f"Translated text saved to {output_file}."
        else:
            return "No translation found."
    else:
        return f"API Error: {response.status_code} - {response.text}"

In [8]:

# Example usage
api_key = "your_key"  # Replace this with your actual DeepL API key
translated_text = translate_text_with_deepl("transcription_output.txt", api_key)
